In [ ]:
# import requests, os, sys, tarfile
# from tqdm import tqdm

# url = "http://labrosa.ee.columbia.edu/~dpwe/tmp/millionsongsubset.tar.gz"
# archive_name = "millionsongsubset.tar.gz"
# folder_name = "millionsongsubset"

# def download_file(url, archive_name):
#     response = requests.get(url, stream=True)
#     total_size_in_bytes= int(response.headers.get('content-length', 0))
#     block_size = 1024
#     progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
#     with open(archive_name, 'wb') as file:
#         for data in response.iter_content(block_size):
#             progress_bar.update(len(data))
#             file.write(data)
#     progress_bar.close()
#     if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
#         print("ERROR, something went wrong")
#         sys.exit(1)

# def extract_file(archive_name, folder_name):
#     with tarfile.open(archive_name, 'r:gz') as tar:
#         tar.extractall(folder_name)

# if not os.path.exists(folder_name):
#     download_file(url, archive_name)
# if not os.path.exists(folder_name):
#     extract_file(archive_name, folder_name)

In [ ]:
# # Explore the data
# import h5py
# import numpy as np

# sub_folder = "MillionSongSubset"

# filename = "A/A/A/TRAAAAW128F429D538.h5"
# filename = os.path.join(folder_name, sub_folder, filename)
# h5 = h5py.File(filename, 'r')
# print(h5.keys())
# print(h5['metadata'].keys())
# print(h5['analysis'].keys())

# # Show fields in songs
# # print(list(h5['metadata']['songs'].dtype.fields.keys()))
# # print(list(h5['metadata']['songs']))
# keys = list(h5['metadata']['songs'].dtype.fields.keys())
# for key in keys:
#     print(key, h5['metadata']['songs'][key])
# # Show song id
# print(h5['metadata']['songs']['song_id'][0])

<KeysViewHDF5 ['analysis', 'metadata', 'musicbrainz']>
<KeysViewHDF5 ['artist_terms', 'artist_terms_freq', 'artist_terms_weight', 'similar_artists', 'songs']>
<KeysViewHDF5 ['bars_confidence', 'bars_start', 'beats_confidence', 'beats_start', 'sections_confidence', 'sections_start', 'segments_confidence', 'segments_loudness_max', 'segments_loudness_max_time', 'segments_loudness_start', 'segments_pitches', 'segments_start', 'segments_timbre', 'songs', 'tatums_confidence', 'tatums_start']>
analyzer_version [b'']
artist_7digitalid [165270]
artist_familiarity [0.58179377]
artist_hotttnesss [0.40199754]
artist_id [b'ARD7TVE1187B99BFB1']
artist_latitude [nan]
artist_location [b'California - LA']
artist_longitude [nan]
artist_mbid [b'e77e51a5-4761-45b3-9847-2051f811e366']
artist_name [b'Casual']
artist_playmeid [4479]
genre [b'']
idx_artist_terms [0]
idx_similar_artists [0]
release [b'Fear Itself']
release_7digitalid [300848]
song_hotttnesss [0.60211999]
song_id [b'SOMZWCG12A8C13C480']
title [

In [ ]:
# # Walk through all files and add song_id and features to a pandas dataframe
# import pandas as pd
# import glob
# from tqdm import tqdm

# def get_dataframe(basedir, ext='.h5'):
#     metadata_data = []
#     analysis_data = []
#     for root, _, files in tqdm(os.walk(basedir), total=932):
#         files = glob.glob(os.path.join(root, f'*{ext}'))
#         for file in files:
#             h5 = h5py.File(file, 'r')
#             metadata = h5['metadata']
#             keys = list(metadata['songs'].dtype.fields.keys())
#             assert len(metadata['songs']) == 1
#             values = list(metadata['songs'][0])
#             row = dict(zip(keys, values))
#             row['song_id'] = row['song_id'].decode('utf-8')
#             song_id = row['song_id']
#             metadata_data.append(row)

#             analysis = h5['analysis']
#             keys = list(analysis['songs'].dtype.fields.keys())
#             values = list(analysis['songs'][0])
#             row = dict(zip(keys, values))
#             row['song_id'] = song_id
#             analysis_data.append(row)
#             h5.close()

#     metadata_data = pd.DataFrame(metadata_data)
#     analysis_data = pd.DataFrame(analysis_data)
#     # Move song_id to the first column
#     metadata_data = metadata_data[['song_id'] + [col for col in metadata_data.columns if col != 'song_id']]
#     analysis_data = analysis_data[['song_id'] + [col for col in analysis_data.columns if col != 'song_id']]
#     return metadata_data, analysis_data

# metadata_df_path = "metadata.csv"
# analysis_df_path = "analysis.csv"
# path = os.path.join(folder_name, sub_folder)

# if not os.path.exists(metadata_df_path) or not os.path.exists(analysis_df_path):
#     print("Walking through directory", path)
#     metadata_df, analysis_df = get_dataframe(folder_name)
#     metadata_df.to_csv(metadata_df_path)
#     analysis_df.to_csv(analysis_df_path)
# else:
#     metadata_df = pd.read_csv(metadata_df_path)
#     analysis_df = pd.read_csv(analysis_df_path)

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

# Load train triplets
filename = "train_triplets.txt"

def get_triplets(filename):
    triplets = []
    with open(filename, 'r') as file:
        for line in tqdm(file, total=48373586):
            user_id, song_id, play_count = line.strip().split('\t')
            triplets.append((user_id, song_id, int(play_count)))
    return triplets

triplets_path = "triplets.csv"
if not os.path.exists(triplets_path):
    triplets = get_triplets(filename)
    triplets_df = pd.DataFrame(triplets, columns=['user_id', 'song_id', 'play_count'])
    triplets_df.to_csv(triplets_path)
else:
    triplets_df = pd.read_csv(triplets_path)

In [ ]:
from model import EASE

model = EASE()
# Rename the columns to user_id and item_id and ratings
triplets_df.columns = ['user_id', 'item_id', 'rating']
# Use a subset of the data
N = 10000
triplets_df = triplets_df[:N]
model.fit(triplets_df, implicit=False)

In [ ]:
# Measure the performance of the model

# def predict(self, train, users, items, k):
#         items = self.item_enc.transform(items)
#         dd = train.loc[train.user_id.isin(users)]
#         dd['ci'] = self.item_enc.transform(dd.item_id)
#         dd['cu'] = self.user_enc.transform(dd.user_id)
#         g = dd.groupby('cu')
#         with Pool(cpu_count()) as p:
#             user_preds = p.starmap(
#                 self.predict_for_user,
#                 [(user, group, self.pred[user, :], items, k) for user, group in g],
#             )
#         df = pd.concat(user_preds)
#         df['item_id'] = self.item_enc.inverse_transform(df['item_id'])
#         df['user_id'] = self.user_enc.inverse_transform(df['user_id'])
#         return df

def precision_at_k(predictions, k):
    predictions = predictions.sort_values('score', ascending=False)
    predictions = predictions[:k]
    return len(predictions[predictions['score'] > 0]) / k

def recall_at_k(predictions, k):
    predictions = predictions.sort_values('score', ascending=False)
    predictions = predictions[:k]
    return len(predictions[predictions['score'] > 0]) / len(predictions)

def evaluate(model, triplets_df, N=10, k=10):
    users = triplets_df.user_id.unique()
    items = triplets_df.item_id.unique()
    users = users[:N]
    items = items[:N]
    predictions = model.predict(triplets_df, users, items, k)

    precision_scores = []
    recall_scores = []

    for user in users:
        user_predictions = predictions[predictions['user_id'] == user]
        precision_scores.append(precision_at_k(user_predictions, k))
        recall_scores.append(recall_at_k(user_predictions, k))
        
    precision = np.mean(precision_scores)
    recall = np.mean(recall_scores)
    f1 = 2 * (precision * recall) / (precision + recall)
    return precision, recall, f1

precision, recall, f1 = evaluate(model, triplets_df, N=100, k=10)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

d:\MSU\MastersFall\CSE881\project\model.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['ci'] = self.item_enc.transform(dd.item_id)
d:\MSU\MastersFall\CSE881\project\model.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dd['cu'] = self.user_enc.transform(dd.user_id)


Precision: 0.0
Recall: 0.0
F1: nan


C:\Users\ssit5\AppData\Local\Temp\ipykernel_3812\1245215593.py:46: RuntimeWarning: invalid value encountered in double_scalars
  f1 = 2 * (precision * recall) / (precision + recall)


In [ ]:
X = model.X
X

<10000x7849 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [ ]:
model.pred.sum()

0.0

In [ ]:
triplets_df

,Unnamed: 0,user_id,song_id,play_count
0,0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1
...,...,...,...,...
48373581,48373581,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUHHHH12AF729E4AF,2
48373582,48373582,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUJVIT12A8C1451C1,1
48373583,48373583,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOUSMXX12AB0185C24,1
48373584,48373584,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOWYSKH12AF72A303A,3


In [5]:
from torch_ease import TorchEASE

model = TorchEASE(train=triplets_df[:1000], user_col='user_id', item_col='song_id', score_col='play_count')
model.fit()

2024-11-01 01:40:59 [INFO] notebook - Building user + item lookup
2024-11-01 01:40:59 [INFO] notebook - Building item hashmap
2024-11-01 01:40:59 [INFO] notebook - User + item lookup complete
2024-11-01 01:40:59 [INFO] notebook - Sparse data built
2024-11-01 01:40:59 [INFO] notebook - Building G Matrix
2024-11-01 01:40:59 [INFO] notebook - Building B matrix


d:\MSU\MastersFall\CSE881\project\torch_ease.py:63: UserWarning: torch.sparse.SparseTensor(indices, values, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, dtype=, device=). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:631.)
  self.sparse = torch.sparse.FloatTensor(self.indices.t(), self.values)


In [11]:
user_df = triplets_df[:1000][['user_id']].drop_duplicates()
print(user_df)

predictions = model.predict_all(user_df, k=5)
print(predictions)

                                      user_id
0    b80344d063b5ccb3212f76538f3d9e43d87dca9e
104  85c1f87fea955d09b4bec2e36aee110927aedf9a
125  bd4c6e843f00bd476847fb75c47b4fb430a06856
136  8937134734f869debcab8f23d77465b4caaa85df
149  969cc6fb74e076a68e36a04409cb9d3765757508
182  4bd88bfb25263a75bbdd467e74018f4ae570e5df
199  e006b1a48f466bf59feefed32bec6494495a4436
227  9d6f0ead607ac2a6c2460e4d14fb439a146b7dec
238  9bb911319fbc04f01755814cb5edb21df3d1a336
252  b64cdd1a0bd907e5e00b39e345194768e330d652
361  17aa9f6dbdf753831da8f38c71b66b64373de613
498  d6589314c0a9bcbca4fee0c93b14bc402363afea
558  5a905f000fc1ff3df7ca807d57edb608863db05d
2024-11-01 01:42:12 [INFO] notebook - Predictions are made
2024-11-01 01:42:12 [INFO] notebook - Removing owned items
2024-11-01 01:42:12 [INFO] notebook - TopK selected per user
2024-11-01 01:42:12 [INFO] notebook - Predictions are returned to user
                                     user_id  user_id_id  \
0   b80344d063b5ccb3212f76538f3d9e43d87dca9e  